# Bioassay

This dataset is a collection of 21 bioassays (screens) that measure the activity of various compounds against different biological targets.

Highly imbalanced.

In [2]:
library(glmnet)
library(tidyverse)
library(caret)
library(pROC)
library(randomForest)
library(gbm)
library(rpart)
require(rpart.plot)
library(e1071)
library(ranger)

In [4]:
bio <- read.csv("C:/Users/n__gu/Desktop/bio.csv", header=TRUE)

In [5]:
data <- bio %>% sample_frac(0.1)
dim(data)
str(data)

[1] 4783   33

'data.frame':	4783 obs. of  33 variables:
 $ WBN_GC_L_0.25: num  -1.57 -2.3 -1.82 -2.26 -1.58 ...
 $ WBN_GC_H_0.25: num  1.38 1.74 1.51 1.88 1.22 ...
 $ WBN_GC_L_0.50: num  -1.99 -2.56 -2.14 -2.46 -2.01 ...
 $ WBN_GC_H_0.50: num  2.09 2.27 2.08 2.36 1.99 ...
 $ WBN_GC_L_0.75: num  -2.54 -2.95 -2.73 -2.88 -2.77 ...
 $ WBN_GC_H_0.75: num  2.87 2.97 2.9 3.1 2.87 ...
 $ WBN_GC_L_1.00: num  -3.16 -3.46 -3.42 -3.44 -3.6 ...
 $ WBN_GC_H_1.00: num  3.67 3.73 3.78 3.95 3.76 ...
 $ WBN_EN_L_0.25: num  -0.779 -0.789 -0.795 -0.779 -0.778 ...
 $ WBN_EN_H_0.25: num  1.2 1.25 1.33 1.36 1.54 ...
 $ WBN_EN_L_0.50: num  -1.57 -1.58 -1.62 -1.58 -1.58 ...
 $ WBN_EN_H_0.50: num  1.86 1.97 2.12 2.16 2.38 ...
 $ WBN_EN_L_0.75: num  -2.37 -2.38 -2.45 -2.39 -2.4 ...
 $ WBN_EN_H_0.75: num  2.64 2.77 2.99 3.03 3.26 ...
 $ WBN_EN_L_1.00: num  -3.16 -3.18 -3.28 -3.21 -3.23 ...
 $ WBN_EN_H_1.00: num  3.45 3.59 3.88 3.92 4.15 ...
 $ WBN_LP_L_0.25: num  -0.833 -0.847 -0.832 -0.928 -1.081 ...
 $ WBN_LP_H_0.25: num  1.

In [6]:
set.seed(101) # Set Seed so that same sample can be reproduced in future also
# Now Selecting 75% of data as sample from total 'n' rows of the data  

sample <- sample.int(n = nrow(data), size = floor(.75*nrow(data)), replace = F)
train <- data[sample, ]
test  <- data[-sample, ]

## LASSO

In [48]:
head(train)

,WBN_GC_L_0.25,WBN_GC_H_0.25,WBN_GC_L_0.50,WBN_GC_H_0.50,WBN_GC_L_0.75,WBN_GC_H_0.75,WBN_GC_L_1.00,WBN_GC_H_1.00,WBN_EN_L_0.25,WBN_EN_H_0.25,...,WBN_LP_H_1.00,XLogP,PSA,NumRot,NumHBA,NumHBD,MW,BBB,BadGroup,Outcome
2873,-2.40396,1.95283,-2.61228,2.54595,-2.99884,3.40225,-3.67408,4.32266,-0.833599,1.43350,...,3.97767,1.860,115.56,12,6,1,429.473,0,0,Inactive
2143,-1.93453,1.57144,-2.30531,2.21371,-2.79733,3.00279,-3.35643,3.83447,-0.786392,1.40592,...,3.71675,3.244,68.02,5,2,1,327.771,1,0,Inactive
3281,-2.25209,1.84241,-2.52633,2.57568,-3.00032,3.39810,-3.64635,4.27884,-0.852305,1.30931,...,4.16970,5.700,66.23,0,2,1,330.137,1,1,Inactive
3004,-2.38236,1.73678,-2.50541,2.41854,-2.74909,3.16963,-3.19127,3.95260,-0.782188,1.21421,...,3.36988,2.278,71.03,7,6,2,362.470,0,0,Inactive
315,-2.35489,1.47263,-2.61217,2.08603,-2.99875,2.83116,-3.47244,3.62571,-0.771912,1.38378,...,3.54441,3.025,88.34,6,4,2,366.425,0,0,Inactive
3712,-1.53156,1.28867,-2.02812,2.03224,-2.67174,2.89818,-3.38477,3.79508,-0.780526,1.47770,...,4.04090,5.848,61.67,3,1,0,425.355,1,1,Inactive


In [7]:
train <- as.data.frame(train)

x <- as.matrix(train[,1:32])
y <- as.matrix(as.factor(train[,33]))

In [8]:
set.seed(100) 
cv <- cv.glmnet(x, y, nfolds = 3,alpha = 1,family="binomial")

# Display the best lambda value
cv$lambda.min

[1] 0.002155678

In [9]:
model <- glmnet(x, y , alpha = 1, lambda = cv$lambda.min,family="binomial")
# Display regression coefficients
coef(model)

33 x 1 sparse Matrix of class "dgCMatrix"
                      s0
(Intercept)   5.93799598
WBN_GC_L_0.25 .         
WBN_GC_H_0.25 .         
WBN_GC_L_0.50 .         
WBN_GC_H_0.50 .         
WBN_GC_L_0.75 0.18602132
WBN_GC_H_0.75 .         
WBN_GC_L_1.00 0.04133108
WBN_GC_H_1.00 .         
WBN_EN_L_0.25 .         
WBN_EN_H_0.25 .         
WBN_EN_L_0.50 .         
WBN_EN_H_0.50 .         
WBN_EN_L_0.75 .         
WBN_EN_H_0.75 .         
WBN_EN_L_1.00 .         
WBN_EN_H_1.00 .         
WBN_LP_L_0.25 .         
WBN_LP_H_0.25 .         
WBN_LP_L_0.50 .         
WBN_LP_H_0.50 .         
WBN_LP_L_0.75 .         
WBN_LP_H_0.75 .         
WBN_LP_L_1.00 .         
WBN_LP_H_1.00 .         
XLogP         .         
PSA           .         
NumRot        .         
NumHBA        .         
NumHBD        .         
MW            .         
BBB           .         
BadGroup      .         

In [100]:
x.test <- as.matrix(test[,1:32])
predictions <- model %>% predict(x.test) %>% as.vector()

table(test[,33])
auc(roc(response = test$Outcome, predictor = predictions))


  Active Inactive 
       7     1189 

Setting levels: control = Active, case = Inactive
Setting direction: controls < cases


Area under the curve: 0.6111

In [10]:
x.train <- as.matrix(train[,1:32])
predictions <- model %>% predict(x.train) %>% as.vector()

table(train[,33])
auc(roc(response = train$Outcome, predictor = predictions))


  Active Inactive 
      18     3569 

Setting levels: control = Active, case = Inactive
Setting direction: controls < cases


Area under the curve: 0.644

The difference between results, 0.61 for test and 0.64 for train, is negligible. Hence there is no issue of over- or underfitting. However, the performance is not good.

## Decision Tree

In [11]:
control <- trainControl(method = "cv", 10 )
tree <- train(form = Outcome ~ .,
      data = train,
      method = "rpart",  # Decision Tree
      trControl = control,
      tuneGrid = 
        expand.grid(.cp = seq(.01,0.5))) 

In [12]:
tree

CART 

3587 samples
  32 predictor
   2 classes: 'Active', 'Inactive' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 3228, 3228, 3229, 3228, 3228, 3228, ... 
Resampling results:

  Accuracy  Kappa
  0.994983  0    

Tuning parameter 'cp' was held constant at a value of 0.01

In [14]:
tree1 <- rpart(Outcome~., data=train,control = rpart.control(minbucket =3, cp= 0.01))
predict <- predict(tree1, newdata=test,type="class")
confusionMatrix(test$Outcome,predict)

Confusion Matrix and Statistics

          Reference
Prediction Active Inactive
  Active        0       10
  Inactive      0     1186
                                         
               Accuracy : 0.9916         
                 95% CI : (0.9847, 0.996)
    No Information Rate : 1              
    P-Value [Acc > NIR] : 1.000000       
                                         
                  Kappa : 0              
                                         
 Mcnemar's Test P-Value : 0.004427       
                                         
            Sensitivity :       NA       
            Specificity : 0.991639       
         Pos Pred Value :       NA       
         Neg Pred Value :       NA       
             Prevalence : 0.000000       
         Detection Rate : 0.000000       
   Detection Prevalence : 0.008361       
      Balanced Accuracy :       NA       
                                         
       'Positive' Class : Active         
                          

In [15]:
tree1 <- rpart(Outcome~., data=train,control = rpart.control(minbucket =10, cp= 0.01))
predict <- predict(tree1, newdata=test,type="class")
confusionMatrix(test$Outcome,predict)

Confusion Matrix and Statistics

          Reference
Prediction Active Inactive
  Active        0       10
  Inactive      0     1186
                                         
               Accuracy : 0.9916         
                 95% CI : (0.9847, 0.996)
    No Information Rate : 1              
    P-Value [Acc > NIR] : 1.000000       
                                         
                  Kappa : 0              
                                         
 Mcnemar's Test P-Value : 0.004427       
                                         
            Sensitivity :       NA       
            Specificity : 0.991639       
         Pos Pred Value :       NA       
         Neg Pred Value :       NA       
             Prevalence : 0.000000       
         Detection Rate : 0.000000       
   Detection Prevalence : 0.008361       
      Balanced Accuracy :       NA       
                                         
       'Positive' Class : Active         
                          

In [16]:
tree1 <- rpart(Outcome~., data=train,control = rpart.control(minbucket =15, cp= 0.01))
predict <- predict(tree1, newdata=test,type="class")
confusionMatrix(test$Outcome,predict)

Confusion Matrix and Statistics

          Reference
Prediction Active Inactive
  Active        0       10
  Inactive      0     1186
                                         
               Accuracy : 0.9916         
                 95% CI : (0.9847, 0.996)
    No Information Rate : 1              
    P-Value [Acc > NIR] : 1.000000       
                                         
                  Kappa : 0              
                                         
 Mcnemar's Test P-Value : 0.004427       
                                         
            Sensitivity :       NA       
            Specificity : 0.991639       
         Pos Pred Value :       NA       
         Neg Pred Value :       NA       
             Prevalence : 0.000000       
         Detection Rate : 0.000000       
   Detection Prevalence : 0.008361       
      Balanced Accuracy :       NA       
                                         
       'Positive' Class : Active         
                          

In [17]:
predict <- predict(tree1, newdata=train,type="class")
confusionMatrix(train$Outcome,predict)

Confusion Matrix and Statistics

          Reference
Prediction Active Inactive
  Active        0       18
  Inactive      0     3569
                                         
               Accuracy : 0.995          
                 95% CI : (0.9921, 0.997)
    No Information Rate : 1              
    P-Value [Acc > NIR] : 1              
                                         
                  Kappa : 0              
                                         
 Mcnemar's Test P-Value : 6.151e-05      
                                         
            Sensitivity :       NA       
            Specificity : 0.994982       
         Pos Pred Value :       NA       
         Neg Pred Value :       NA       
             Prevalence : 0.000000       
         Detection Rate : 0.000000       
   Detection Prevalence : 0.005018       
      Balanced Accuracy :       NA       
                                         
       'Positive' Class : Active         
                          

As we observe from the accuracy at 0.99 for test data, this model seems to perform very well. However, it predicted all instances as inactive, since the data is highly imbalanced. When we predict the train data, same problem occurs. 

## Random Forest

In [18]:
folds <- createMultiFolds(train$Outcome, k = 10, times = 3)
control <- trainControl(method = "cv", number = 10, verboseIter = TRUE,
                        classProbs = TRUE, savePredictions = TRUE, index = folds, allowParallel = TRUE)
tune_grid <- expand.grid(mtry = c(2,5,7,10), splitrule = "gini", min.node.size = 5)
rf_gridsearch <- train(Outcome~., train, method = "ranger", tuneGrid = tune_grid, trControl = control)

+ Fold01.Rep1: mtry= 2, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry= 2, splitrule=gini, min.node.size=5 
+ Fold01.Rep1: mtry= 5, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry= 5, splitrule=gini, min.node.size=5 
+ Fold01.Rep1: mtry= 7, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry= 7, splitrule=gini, min.node.size=5 
+ Fold01.Rep1: mtry=10, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry=10, splitrule=gini, min.node.size=5 
+ Fold02.Rep1: mtry= 2, splitrule=gini, min.node.size=5 
- Fold02.Rep1: mtry= 2, splitrule=gini, min.node.size=5 
+ Fold02.Rep1: mtry= 5, splitrule=gini, min.node.size=5 
- Fold02.Rep1: mtry= 5, splitrule=gini, min.node.size=5 
+ Fold02.Rep1: mtry= 7, splitrule=gini, min.node.size=5 
- Fold02.Rep1: mtry= 7, splitrule=gini, min.node.size=5 
+ Fold02.Rep1: mtry=10, splitrule=gini, min.node.size=5 
- Fold02.Rep1: mtry=10, splitrule=gini, min.node.size=5 
+ Fold03.Rep1: mtry= 2, splitrule=gini, min.node.size=5 
- Fold03.Rep1: mtry= 2, splitru

In [19]:
predictrf <-predict(rf_gridsearch, newdata=test)

In [20]:
confusionMatrix(test$Outcome,predictrf)

Confusion Matrix and Statistics

          Reference
Prediction Active Inactive
  Active        0       10
  Inactive      0     1186
                                         
               Accuracy : 0.9916         
                 95% CI : (0.9847, 0.996)
    No Information Rate : 1              
    P-Value [Acc > NIR] : 1.000000       
                                         
                  Kappa : 0              
                                         
 Mcnemar's Test P-Value : 0.004427       
                                         
            Sensitivity :       NA       
            Specificity : 0.991639       
         Pos Pred Value :       NA       
         Neg Pred Value :       NA       
             Prevalence : 0.000000       
         Detection Rate : 0.000000       
   Detection Prevalence : 0.008361       
      Balanced Accuracy :       NA       
                                         
       'Positive' Class : Active         
                          

In [21]:
predictrf <-predict(rf_gridsearch, newdata=train)
confusionMatrix(train$Outcome,predictrf)

Confusion Matrix and Statistics

          Reference
Prediction Active Inactive
  Active        8       10
  Inactive      0     3569
                                          
               Accuracy : 0.9972          
                 95% CI : (0.9949, 0.9987)
    No Information Rate : 0.9978          
    P-Value [Acc > NIR] : 0.816108        
                                          
                  Kappa : 0.6142          
                                          
 Mcnemar's Test P-Value : 0.004427        
                                          
            Sensitivity : 1.000000        
            Specificity : 0.997206        
         Pos Pred Value : 0.444444        
         Neg Pred Value : 1.000000        
             Prevalence : 0.002230        
         Detection Rate : 0.002230        
   Detection Prevalence : 0.005018        
      Balanced Accuracy : 0.998603        
                                          
       'Positive' Class : Active          
      

Random Forest also has accuracy of 0.9916 for test data, however this model managed to catch some active level in the training data, but at accuracy level of 0.9972, it has overfitting issue.

## Stochastic Gradient Boosting

In [22]:
folds <- createMultiFolds(train$Outcome, k = 10, times = 1)
control <- trainControl(method = "cv", number = 10, verboseIter = TRUE, summaryFunction = twoClassSummary,
                        classProbs = TRUE, savePredictions = TRUE, index = folds, allowParallel = TRUE)
tune_grid <- expand.grid(interaction.depth=c(1, 3, 5), n.trees = c(500, 1000), shrinkage=c(0.01, 0.001), n.minobsinnode = 10)
gbm_gridsearch <- train(Outcome ~., data = train, method = "gbm", metric = "Accuracy", tuneGrid = tune_grid, trControl = control)

Warning message in train.default(x, y, weights = w, ...):
"The metric "Accuracy" was not in the result set. ROC will be used instead."

+ Fold01.Rep1: shrinkage=0.001, interaction.depth=1, n.minobsinnode=10, n.trees=1000 
Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        0.0657             nan     0.0010   -0.0000
     2        0.0657             nan     0.0010   -0.0000
     3        0.0657             nan     0.0010   -0.0000
     4        0.0657             nan     0.0010   -0.0000
     5        0.0657             nan     0.0010   -0.0000
     6        0.0657             nan     0.0010   -0.0000
     7        0.0657             nan     0.0010   -0.0000
     8        0.0656             nan     0.0010   -0.0000
     9        0.0656             nan     0.0010   -0.0000
    10        0.0656             nan     0.0010   -0.0000
    20        0.0655             nan     0.0010   -0.0000
    40        0.0653             nan     0.0010   -0.0000
    60        0.0651             nan     0.0010   -0.0000
    80        0.0650             nan     0.0010   -0.0000
   100        0.0649             nan     0.0

In [23]:
predictions <- predict(gbm_gridsearch, newdata = test)
confusionMatrix(test$Outcome,predictions)

Confusion Matrix and Statistics

          Reference
Prediction Active Inactive
  Active        0       10
  Inactive      0     1186
                                         
               Accuracy : 0.9916         
                 95% CI : (0.9847, 0.996)
    No Information Rate : 1              
    P-Value [Acc > NIR] : 1.000000       
                                         
                  Kappa : 0              
                                         
 Mcnemar's Test P-Value : 0.004427       
                                         
            Sensitivity :       NA       
            Specificity : 0.991639       
         Pos Pred Value :       NA       
         Neg Pred Value :       NA       
             Prevalence : 0.000000       
         Detection Rate : 0.000000       
   Detection Prevalence : 0.008361       
      Balanced Accuracy :       NA       
                                         
       'Positive' Class : Active         
                          

In [24]:
predictions <- predict(gbm_gridsearch, newdata = train)
confusionMatrix(train$Outcome,predictions)

Confusion Matrix and Statistics

          Reference
Prediction Active Inactive
  Active        0       18
  Inactive      0     3569
                                         
               Accuracy : 0.995          
                 95% CI : (0.9921, 0.997)
    No Information Rate : 1              
    P-Value [Acc > NIR] : 1              
                                         
                  Kappa : 0              
                                         
 Mcnemar's Test P-Value : 6.151e-05      
                                         
            Sensitivity :       NA       
            Specificity : 0.994982       
         Pos Pred Value :       NA       
         Neg Pred Value :       NA       
             Prevalence : 0.000000       
         Detection Rate : 0.000000       
   Detection Prevalence : 0.005018       
      Balanced Accuracy :       NA       
                                         
       'Positive' Class : Active         
                          

SGB also cannot detect the imbalanced level in this dataset, as the accuracy level is ove 0.99 for both test and train data.

Overall comparison of methods: LASSO accuracy was quite low and the other methods failed to catch the imbalanced class. Only random forest catched from the training data. Based on the results, none of the methods worked at the desirable level as the data awas extremely imbalanced. With some necessary modifications such as collecting more data, random forest might work well.